In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn import set_config
set_config(print_changed_only=False)

In [ ]:
df=pd.read_csv("../input/insurance/insurance.csv")
df.head()

In [ ]:
#Cinsiyet - sigara içme ve bölge verilerini standardizasyon işlemlerinden geçirmemiz gerekir. Bu noktada

In [ ]:
ada=df.copy()

In [ ]:
dmy=pd.get_dummies(ada[["sex","smoker"]])
dmy.head()

In [ ]:
lbe=LabelEncoder()
ada["region"]=lbe.fit_transform(ada["region"])

In [ ]:
ada.head()

In [ ]:
ada["region"]

In [ ]:
y=ada["charges"]

In [ ]:
X=ada.drop(["sex","smoker","charges"],axis=1)

In [ ]:
X=pd.concat([X,dmy[["sex_male","smoker_yes"]]],axis=1)
Xtrain,Xtest,ytrain,ytest=train_test_split(X,y,test_size=0.25,random_state=42)

In [ ]:
#Lineer Regresyon
from sklearn.linear_model import LinearRegression
lm=LinearRegression()
model=lm.fit(Xtrain,ytrain)
ytahmin=model.predict(Xtest)
np.sqrt(mean_squared_error(ytest,ytahmin))

In [ ]:
#KNN Modeli

In [ ]:
knn_model=KNeighborsRegressor()
knn_model.fit(Xtrain,ytrain)
ytahmin=knn_model.predict(Xtest)
np.sqrt(mean_squared_error(ytest,ytahmin))

In [ ]:
#pd.DataFrame({"Gerçek Test":ytest,"Tahmin Edilen":knn_model.predict(Xtest)})

In [ ]:
#KNN Modelinde Model Tuning
knn_model

In [ ]:
knn_parametre={"n_neighbors":np.arange(1,25,1)}
knn=KNeighborsRegressor()
knn_cv=GridSearchCV(knn,knn_parametre,cv=10)

In [ ]:
knn_cv.fit(Xtrain,ytrain)

In [ ]:
knn_cv.best_params_

In [ ]:
knn_tuned=KNeighborsRegressor(n_neighbors=21)
knn_tuned.fit(Xtrain,ytrain)
tunedtahmin=knn_tuned.predict(Xtest)
np.sqrt(mean_squared_error(ytest,tunedtahmin))

In [ ]:
#KNN modeli veri setimiz için istediğimiz kadar etkili olamadı

In [ ]:
#CART Modeli
from sklearn.tree import DecisionTreeRegressor
cart_model=DecisionTreeRegressor()
cart_model.fit(Xtrain,ytrain)
ytahmin=cart_model.predict(Xtest)
np.sqrt(mean_squared_error(ytest,ytahmin))

In [ ]:
#CART yapısının hiperparametre değerlerini incelemek için 
cart_model

In [ ]:
#Model Tuning
cart_parametre={"min_samples_split": range(2,200),
              "max_leaf_nodes":range(2,10)}
cart=DecisionTreeRegressor()
cart_cv=GridSearchCV(cart,cart_parametre,cv=10,n_jobs=-1)
cart_cv.fit(Xtrain,ytrain)

In [ ]:
cart_cv.best_params_

In [ ]:
cart_tuned=DecisionTreeRegressor(max_leaf_nodes=9,min_samples_split=68)
cart_tuned.fit(Xtrain,ytrain)
tuned_tahmin=cart_tuned.predict(Xtest)
np.sqrt(mean_squared_error(ytest,tuned_tahmin))

In [ ]:
#Hiperparametreler ile oynanması test verileri ile tahminde hata kareler ortalamasını 5k civarına kadar çekebildi

In [ ]:
#Random Forest Regresyonu

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf_model=RandomForestRegressor()
rf_model.fit(Xtrain,ytrain)
ytahmin=rf_model.predict(Xtest)
np.sqrt(mean_squared_error(ytest,ytahmin))

In [ ]:
rf_parametre={"max_depth":list(range(1,10)),
            "n_estimators":[200,500,700]}
random_forest=RandomForestRegressor()
rf_cv=GridSearchCV(random_forest,rf_parametre,cv=10,n_jobs=-1)

In [ ]:
rf_cv.fit(Xtrain,ytrain)
rf_cv.best_params_

In [ ]:
rf_tuned=RandomForestRegressor(max_depth=4,n_estimators=500)
rf_tuned.fit(Xtrain,ytrain)
tuned_tahmin=rf_tuned.predict(Xtest)
np.sqrt(mean_squared_error(ytest,tuned_tahmin))

In [ ]:
#GBM-Gradient Boosting Machine
from sklearn.ensemble import GradientBoostingRegressor
gbm_model=GradientBoostingRegressor()
gbm_model.fit(Xtrain,ytrain)
ytahmin=gbm_model.predict(Xtest)
np.sqrt(mean_squared_error(ytest,ytahmin))

In [ ]:
#Model Tuning 
gbm_parametre={"learning_rate":[0.001,0.01,0.1,0.2],
     "max_depth":[3,5,8,50,100],
     "n_estimators":[150,200,500,700,1200],
     "subsample":[1,0.5,0.75]}
gbm=GradientBoostingRegressor()
gbm_cv=GridSearchCV(gbm,gbm_parametre,cv=10,n_jobs=-1)
gbm_cv.fit(Xtrain,ytrain)
gbm_cv.best_params_

In [ ]:
gbm_tuned=GradientBoostingRegressor(learning_rate=0.01,max_depth=3,n_estimators=500,subsample=0.75)
gbm_tuned.fit(Xtrain,ytrain)
tuned_tahmin=gbm_tuned.predict(Xtest)
np.sqrt(mean_squared_error(ytest,tuned_tahmin))

In [ ]:
#Model tuning yapılırken hiperparametre seçimi çıkan sonuçlara göre değişiklik gösterebilir. Örneği yukarıda ki örnekte subsapmle için GridSearc benim verdiğim değerlerden maksimum değeri seçmiş
#Bu noktada istenirse gbm_parametre üzerinden gerekli değişiklikler yapılabilir.

In [ ]:
!pip install xgboost

In [ ]:
import xgboost as xgb

In [ ]:
dm_train=xgb.DMatrix(data=Xtrain,label=ytrain)
dm_test=xgb.DMatrix(data=Xtest,label=ytest)

In [ ]:
from xgboost import XGBRegressor

In [ ]:
xgb_model=XGBRegressor()
xgb_model.fit(Xtrain,ytrain)

In [ ]:
ytahmin=xgb_model.predict(Xtest)
np.sqrt(mean_squared_error(ytest,ytahmin))

In [ ]:
xgbparametre={
    "colsample_bytree":[0.4,0.5,0.6,0.9],
    "n_estimators":[100,200,500,1000],
    "max_depth":[2,3,4,5,6],
    "learning_rate":[0.1,0.01,0.5]
}
xgbcvmodel=XGBRegressor()
xgbcvmodel=GridSearchCV(xgbcvmodel,xgbparametre,cv=10,n_jobs=-1,verbose=2)
xgbcvmodel.fit(Xtrain,ytrain)

In [ ]:
xgbcvmodel.best_params_

In [ ]:
tuned_xgb=XGBRegressor(colsample_bytree=0.9,learning_rate=0.01,max_depth=2,n_estimators=1000)
tuned_xgb.fit(Xtrain,ytrain)
tuned_tahmin=tuned_xgb.predict(Xtest)
np.sqrt(mean_squared_error(ytest,tuned_tahmin))

In [ ]:
#En iyi olasılıkları GBM ile yakaladık. Tahmin değerlerini göstermek istersek :
pd.DataFrame({"Y-Test Değerleri":ytest,"Model Tahmnileri":gbm_tuned.predict(Xtest)})